# Purpose
extract masks from cloth images

In [1]:
import os
from pathlib import Path

import numpy as np
import math

import matplotlib.pyplot as plt

import wandb
from wandb.keras import WandbCallback

from PIL import Image

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
from keras import models

# Path list

In [2]:
image_paths, mask_paths = [], []
datapath = '../cloth_segmentation/raw_data/'

for dirpath, _, filename in os.walk(datapath):
    for file in filename:
        path = Path(dirpath).joinpath(file)
        if path.parent.name == 'IMAGES':
            image_paths.append(str(path))
        else: mask_paths.append(str(path))
image_paths = np.asarray(image_paths)
mask_paths = np.asarray(mask_paths)

In [3]:
image_paths, len(image_paths)

(array(['..\\cloth_segmentation\\raw_data\\IMAGES\\img_0001.jpeg',
        '..\\cloth_segmentation\\raw_data\\IMAGES\\img_0002.jpeg',
        '..\\cloth_segmentation\\raw_data\\IMAGES\\img_0003.jpeg',
        '..\\cloth_segmentation\\raw_data\\IMAGES\\img_0004.jpeg',
        '..\\cloth_segmentation\\raw_data\\IMAGES\\img_0005.jpeg',
        '..\\cloth_segmentation\\raw_data\\IMAGES\\img_0006.jpeg',
        '..\\cloth_segmentation\\raw_data\\IMAGES\\img_0007.jpeg',
        '..\\cloth_segmentation\\raw_data\\IMAGES\\img_0008.jpeg',
        '..\\cloth_segmentation\\raw_data\\IMAGES\\img_0009.jpeg',
        '..\\cloth_segmentation\\raw_data\\IMAGES\\img_0010.jpeg',
        '..\\cloth_segmentation\\raw_data\\IMAGES\\img_0011.jpeg',
        '..\\cloth_segmentation\\raw_data\\IMAGES\\img_0012.jpeg',
        '..\\cloth_segmentation\\raw_data\\IMAGES\\img_0013.jpeg',
        '..\\cloth_segmentation\\raw_data\\IMAGES\\img_0014.jpeg',
        '..\\cloth_segmentation\\raw_data\\IMAGES\\img_0015.jp

In [4]:
mask_paths, len(mask_paths)

(array(['..\\cloth_segmentation\\raw_data\\MASKS\\seg_0001.png',
        '..\\cloth_segmentation\\raw_data\\MASKS\\seg_0002.png',
        '..\\cloth_segmentation\\raw_data\\MASKS\\seg_0003.png',
        '..\\cloth_segmentation\\raw_data\\MASKS\\seg_0004.png',
        '..\\cloth_segmentation\\raw_data\\MASKS\\seg_0005.png',
        '..\\cloth_segmentation\\raw_data\\MASKS\\seg_0006.png',
        '..\\cloth_segmentation\\raw_data\\MASKS\\seg_0007.png',
        '..\\cloth_segmentation\\raw_data\\MASKS\\seg_0008.png',
        '..\\cloth_segmentation\\raw_data\\MASKS\\seg_0009.png',
        '..\\cloth_segmentation\\raw_data\\MASKS\\seg_0010.png',
        '..\\cloth_segmentation\\raw_data\\MASKS\\seg_0011.png',
        '..\\cloth_segmentation\\raw_data\\MASKS\\seg_0012.png',
        '..\\cloth_segmentation\\raw_data\\MASKS\\seg_0013.png',
        '..\\cloth_segmentation\\raw_data\\MASKS\\seg_0014.png',
        '..\\cloth_segmentation\\raw_data\\MASKS\\seg_0015.png',
        '..\\cloth_segmen

# Data splits

In [5]:
x_train, x_test, y_train, y_test = train_test_split(image_paths, mask_paths, train_size=0.8)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, train_size=0.5)

# Data input generator

In [6]:
class ImageGenerator(keras.utils.Sequence):
    def __init__(self, x, y, batch_size):
        self.x, self.y = np.asarray(x), np.asarray(y)
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        batch_x = [np.asarray(Image.open(path).resize((512, 512), resample=Image.BICUBIC)) for path in batch_x]
        batch_x = np.asarray(batch_x)

        batch_y = [np.asarray(Image.open(path).resize((512, 512), resample=Image.BICUBIC)) for path in batch_y]
        batch_y = np.asarray(batch_y)
        batch_y[batch_y > 0] = 1
        batch_y = np.asarray(batch_y, dtype=np.float32)

        return batch_x, batch_y

    def on_epoch_end(self):
        indices = np.random.permutation(len(self.x))
        self.x, self.y = self.x[indices], self.y[indices]

In [7]:
train_gen = ImageGenerator(x_train, y_train, 2)
val_gen = ImageGenerator(x_val, y_val, 2)
test_gen = ImageGenerator(x_test, y_test, 2)

In [8]:
del image_paths
del mask_paths

# Wandb

In [9]:
wandb.init(project='cloth_segmentation')

wandb: Currently logged in as: porpoising (use `wandb login --relogin` to force relogin)


# Dice loss and metric

In [10]:
def dice(y_true, y_pred):
    y_true_c = tf.keras.backend.flatten(y_true)
    y_pred_c = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_c * y_pred_c, axis=-1)
    return (2. * intersection + tf.keras.backend.epsilon()) / (
            tf.keras.backend.sum(y_true_c, axis=-1) +
            tf.keras.backend.sum(y_pred_c, axis=-1) +
            tf.keras.backend.epsilon())


def mse_log_dice(y_true, y_pred):
    mse_walls = tf.keras.losses.mean_squared_error(y_true, y_pred)
    dice_walls = tf.keras.backend.log(dice(y_true, y_pred))
    return mse_walls - dice_walls

# Setup

## Keras callback

In [9]:
checkpoint_filepath = '../cloth_segmentation/artifacts/w-{epoch:02d}-{val_loss:.2f}.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_dice',
    mode='max',
    save_best_only=True)

In [14]:
def conv_block(filters, kernel_size, pool_size=1, strides=1):
    return models.Sequential([
        layers.Conv2D(filters=filters, kernel_size=kernel_size, padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        # layers.Dropout(0.2),
        layers.MaxPooling2D(pool_size=pool_size, strides=strides, padding='same')
        ])

## Model

In [15]:
input = layers.Input(shape=(512, 512, 3))
conv_512x512 = conv_block(16, 3)(input)
conv_256x256 = conv_block(32, 3, 2, 2)(conv_512x512)
conv_128x128 = conv_block(64, 3, 2, 2)(conv_256x256)
conv_64x64 = conv_block(128, 3, 2, 2)(conv_128x128)
conv_32x32 = conv_block(256, 3, 2, 2)(conv_64x64)
conv_16x16 = conv_block(512, 3, 2, 2)(conv_32x32)
# conv_8x8 = conv_block(1024, 3, 2)(conv_16x16)

# up_16x16 = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(conv_8x8)
# conv_16x16_up = conv_block(512, 3)(up_16x16)
# add_16 = layers.Add()([conv_16x16_up, conv_16x16])

up_32x32 = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(conv_16x16)
conv_32x32_up = conv_block(256, 3)(up_32x32)
add_32 = layers.Add()([conv_32x32_up, conv_32x32])

up_64x64 = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(add_32)
conv_64x64_up = conv_block(128, 3)(up_64x64)
add_64 = layers.Add()([conv_64x64_up, conv_64x64])

up_128x128 = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(add_64)
conv_128x128_up = conv_block(64, 3)(up_128x128)
add_128 = layers.Add()([conv_128x128_up, conv_128x128])

up_256x256 = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(add_128)
conv_256x256_up = conv_block(32, 3)(up_256x256)
add_256 = layers.Add()([conv_256x256_up, conv_256x256])

up_512x512 = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(add_256)
conv_512x512_up = conv_block(16, 3)(up_512x512)

final_conv = conv_block(1, 1)(conv_512x512_up)
head = layers.Activation("sigmoid")(final_conv)

model = keras.Model(input, head)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 sequential_1 (Sequential)      (None, 512, 512, 16  512         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 sequential_2 (Sequential)      (None, 256, 256, 32  4768        ['sequential_1[0][0]']           
                                )                                                             

In [16]:
model.compile(
    optimizer=keras.optimizers.Adam(0.0001),
    loss=mse_log_dice,
    metrics=[dice]
)

In [17]:
model.fit(x=train_gen, validation_data=val_gen,
          epochs=50, callbacks=[model_checkpoint_callback, WandbCallback()])

Epoch 1/50
200/200 [==============================] - 42s 130ms/step - loss: 1.0292 - dice: 0.4492 - val_loss: 1.2312 - val_dice: 0.3697 - _timestamp: 1648793875.0000 - _runtime: 300.0000
Epoch 2/50
200/200 [==============================] - 26s 131ms/step - loss: 0.9770 - dice: 0.4679 - val_loss: 1.0505 - val_dice: 0.4352 - _timestamp: 1648793901.0000 - _runtime: 326.0000
Epoch 3/50
200/200 [==============================] - 26s 131ms/step - loss: 0.9541 - dice: 0.4766 - val_loss: 1.0103 - val_dice: 0.4529 - _timestamp: 1648793928.0000 - _runtime: 353.0000
Epoch 4/50
200/200 [==============================] - 26s 131ms/step - loss: 0.9441 - dice: 0.4808 - val_loss: 0.9677 - val_dice: 0.4729 - _timestamp: 1648793954.0000 - _runtime: 379.0000
Epoch 5/50
200/200 [==============================] - 26s 131ms/step - loss: 0.9301 - dice: 0.4863 - val_loss: 0.9707 - val_dice: 0.4690 - _timestamp: 1648793980.0000 - _runtime: 405.0000
Epoch 6/50
200/200 [==============================] - 26s 13

# MobileNetV2

## Keras callback

In [11]:
checkpoint_filepath = '../cloth_segmentation/artifacts/unet-w-{epoch:02d}-{val_loss:.2f}.h5'
model_checkpoint_callback_unet = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

## Model

In [25]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(512, 512, 3), include_top=False)

layer_names = [
    'block_1_expand_relu',   # 256x256
    'block_3_expand_relu',   # 128x128
    'block_6_expand_relu',   # 64x64
    'block_13_expand_relu',  # 32x32
    'block_16_project',      # 16x16
]
base_model_outputs = [base_model.get_layer(name).output for name in layer_names]

down_stack = tf.keras.Model(inputs=base_model.input, outputs=base_model_outputs)

down_stack.trainable = False

In [31]:
def unet_model(output_channels: int):
    inputs = tf.keras.layers.Input(shape=[512, 512, 3]),

    skips = down_stack(inputs)
    x = skips[-1]
    print(x)
    skips = reversed(skips[:-1])

    filters = [256, 128, 64, 32, 16]
    for filter, skip in zip(filters, skips):
        x = layers.Conv2DTranspose(filter, kernel_size=3, strides=2, padding='same')(x)
        concat = layers.Concatenate()
        x = concat([x, skip])

    last = tf.keras.layers.Conv2DTranspose(
        filters=output_channels, kernel_size=3, strides=2,
         padding='same')
    x = last(x)

    return tf.keras.Model(inputs=inputs, outputs=x)

In [32]:
OUTPUT_CLASSES = 2

unet = unet_model(output_channels=OUTPUT_CLASSES)
unet.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

unet.summary()

KerasTensor(type_spec=TensorSpec(shape=(None, 16, 16, 320), dtype=tf.float32, name=None), name='model_7/block_16_project/Conv2D:0', description="created by layer 'model_7'")
Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 model_7 (Functional)           [(None, 256, 256, 9  1841984     ['input_11[0][0]']               
                                6),                                                               
                                 (None, 128, 128, 1                                               
                

In [33]:
unet.fit(train_gen, epochs=50,
          validation_data=val_gen,
          callbacks=[model_checkpoint_callback_unet, WandbCallback()])

Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "C:\ProgramData\Anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\ProgramData\Anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\ProgramData\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\ProgramData\Anaconda